# QSAR aquatic toxicity

accuracy issues

In [19]:
#importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.impute import SimpleImputer

#Importing Concrete Compressive Strength Dataset
dataset = pd.read_csv('C:/Users/Manan/Desktop/Fall 2019/Machine Learning COMP6321/Project/Machine-Learning-Project/regression-models/QSAR aquatic toxicity/data/qsar_aquatic_toxicity.csv',sep=";")
data = pd.DataFrame(dataset)

In [20]:
# Check if dataset has null values
result = dataset.isna()
print("-> Contains Missing values                 ",end='')
print(result.values.any())
print("-> Total Number of Missing values:         ",end='')
print(result.sum().sum())
print("-> Number of Missing values by column")
print(result.sum())

-> Contains Missing values                 False
-> Total Number of Missing values:         0
-> Number of Missing values by column
0        0
0.1      0
0.2      0
2.419    0
1.225    0
0.667    0
0.3      0
0.4      0
3.740    0
dtype: int64


In [21]:
X = data.iloc[:, :8].values
Y = data.iloc[:, 8].values

print(data)
print(X.shape)
print(Y.shape)

          0      0.1  0.2  2.419  1.225  0.667  0.3  0.4  3.740
0      0.00    0.000    0  2.638  1.401  0.632    0    0  4.330
1      9.23   11.000    0  5.799  2.930  0.486    0    0  7.019
2      9.23   11.000    0  5.453  2.887  0.495    0    0  6.723
3      9.23   11.000    0  4.068  2.758  0.695    0    0  5.979
4    215.34  327.629    3  0.189  4.677  1.333    0    4  6.064
..      ...      ...  ...    ...    ...    ...  ...  ...    ...
540   24.06   35.776    2  3.326  2.837  0.849    2    0  4.651
541    9.23   11.000    0  3.275  2.727  0.874    0    0  3.953
542    0.00    0.000    0  5.165  3.111  0.732    0    0  6.219
543   13.14    9.507    0  2.859  2.614  0.827    0    0  4.995
544    0.00    0.000    0  2.255  1.800  0.917    0    0  2.480

[545 rows x 9 columns]
(545, 8)
(545,)


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 0)

In [23]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler().fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

In [25]:
# Model Accuracy
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#Defining kernel for GaussianProcessRegressor
#kernel=None would take less time to train, but would give 93.7948904411872 accuracy
kernel = DotProduct() + WhiteKernel()

names = ['SVR', 'DecisionTreeRegressor', 'RandomForestRegressor', 'AdaBoostRegressor','GaussianProcessRegressor','LinearRegression','MLPRegressior']
models = [SVR(),
          DecisionTreeRegressor(),
          RandomForestRegressor(),
          AdaBoostRegressor(),
          GaussianProcessRegressor(kernel=kernel),
          LinearRegression(),
          MLPRegressor()]

param_distributions = {
    'SVR': {'kernel': ['linear', 'rbf','poly'],'C':[1.5, 10]},
    'DecisionTreeRegressor': {'max_depth': [10, 20, 90, 100, None],'min_samples_leaf': [1, 2, 4],'min_samples_split': [2, 5, 10]},
    'RandomForestRegressor': {'max_depth': [10, 20, 90, 100, None],'n_estimators': [16, 200, 600, 800],'bootstrap': [True, False],},
    'AdaBoostRegressor': {'n_estimators': [16, 200, 600, 800],'learning_rate' : [0.01,0.05,0.1,0.3,1],'loss' : ['linear', 'square', 'exponential']},
    'GaussianProcessRegressor': {'normalize_y': ['True','False']},
    'LinearRegression': {},
    'MLPRegressior': {'hidden_layer_sizes': [(100,),(30,20)],'activation':['tanh', 'relu'],'learning_rate' : [0.01,0.05,0.1,0.3,1],'max_iter': [100,200]},
}

accuracy= np.zeros(7)

for counter, model in enumerate(models):
    gridcv = GridSearchCV(estimator = model, param_grid = param_distributions[names[counter]], verbose=1,cv=3)
    gridcv.fit(X_train, Y_train)
    grid_accuracy_test = gridcv.best_estimator_.score(X_test, Y_test)
#     grid_accuracy_test  = sklearn.metrics.accuracy_score(Y_test,  gridcv.best_estimator_.predict(X_test))
    accuracy[counter] = grid_accuracy_test*100
    print("Accuracy for " + names[counter] + ":",accuracy[counter])

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Manan\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Manan\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Manan\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning

Accuracy for SVR: 50.05926337315665
Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:    0.3s finished
C:\Users\Manan\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy for DecisionTreeRegressor: 13.443160351223849
Fitting 3 folds for each of 40 candidates, totalling 120 fits


...

Accuracy for AdaBoostRegressor: 41.893814484035275
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.6s finished


Accuracy for GaussianProcessRegressor: 44.97571828369435
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Accuracy for LinearRegression: 45.00081574176593
Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Manan\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:530: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: learning rate 0.01 is not supported. 

In [ ]:
#Comparision graph between all models
import seaborn as sns
y_pos = np.arange(len(names))
heights = [accuracy[0],accuracy[1],accuracy[2],accuracy[3],accuracy[4],accuracy[5],accuracy[6]]

fig, ax=plt.subplots(1,1,figsize=(12,6))

plt.xticks(rotation='90')
sns.barplot(x=names, y=heights)
plt.ylabel('accuracy score')
plt.title('Wine Quality Dataset model accuracy')